In [ ]:
# Install pymatgen and matminer
!pip install pymatgen
!pip install matminer

     |████████████████████████████████| 3.3MB 5.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 27.4MB 143kB/s 
     |████████████████████████████████| 256kB 47.7MB/s 
     |████████████████████████████████| 296kB 42.7MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 13.2MB 37.7MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
  Using cached https://files.pythonhosted.org/packages/73/ef/8967d406f3f85018ceb5efab50431e901683188f1741ceb053efcab26c87/numpy-1.20.2-cp37-cp37m-manylinux2010_x86_64.whl
     |████████████████████████████████| 552kB 41.2MB/s 
  Created wheel for pymatgen: filename=pymatgen-2022.0.6-cp37-cp37m-linux_x86_64.whl size=3674261 sha256=284029fdbb3be39c42c8a5c012b15bae525c59995fb148aa581dfafd7ffef091
  Stored in directory: /root/.cache/pip/wheel

     |████████████████████████████████| 5.8MB 3.7MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
     |████████████████████████████████| 215kB 40.6MB/s 
     |████████████████████████████████| 829kB 39.9MB/s 
     |████████████████████████████████| 22.3MB 1.5MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
  Created wheel for matminer: filename=matminer-0.6.5-cp37-none-any.whl size=1189789 sha256=6a3b4ae5e2df8f7574bf11baf3e92207119d01e6e3880ebdb877bf13a7f9b49b
  Stored in directory: /root/.cache/pip/wheels/b9/d9/ca/b03a29a28fb675c253a5792995097c48c25b8d409ddac4e2c1
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=eb151430a8549e1a7b011069aeb0c6e7ad48f7303f2e17faf67c068ea1934895
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built matminer future
ERROR: nbclient 0.5.3 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is inc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from tqdm import tqdm
from matminer.datasets import load_dataset
from pymatgen.core.structure import Structure


datasets = ['matbench_jdft2d', 'matbench_phonons', 'matbench_dielectric',
            'matbench_log_kvrh', 'matbench_log_gvrh', 'matbench_perovskites',
            'matbench_mp_gap', 'matbench_mp_is_metal', 'matbench_mp_e_form']
small_datasets = datasets[0:6]
large_datasets = datasets[6:]

def get_neighbors(structure, cutoff):
    neighbors = structure.get_all_neighbors(cutoff)
    return [sorted(nbrs, key=lambda x: x[1]) for nbrs in neighbors]


def cif_parse(dataset):
    df = load_dataset(dataset)
    df = df.values
    all_data = []
    for row in tqdm(df):
        data = {}
        struct = row[0]
        prop = row[1]
        neighbors = get_neighbors(struct, 8)
        for i in neighbors:
            if i != []:
                sites = i
                break

        r = 9
        # while ((len(sites) < 80) or (len(sites) < struct.num_sites)):
        while (len(sites) < 80):
            neighbors = get_neighbors(struct, r)
            sites = neighbors[0]
            r += 1

        supercell = Structure.from_sites(sites[:80])
        data['R'] = supercell.cart_coords
        data['Z'] = np.asarray(supercell.atomic_numbers)
        data['N'] = np.asarray(len(data['R']))
        data['Y'] = np.asarray(prop)
        data['formula'] = struct.formula.replace(' ', '')
        all_data.append(data)
    return all_data

path = 'drive/My Drive/DimeNet/eighty_atoms'
for dataset in large_datasets: # start with small datasets first
    parsed = cif_parse(dataset)
    np.save(f'{path}/{dataset}.npy', parsed)
    print(f'\n{dataset} parsed!\n')

Fetching matbench_mp_gap.json.gz from https://ml.materialsproject.org/projects/matbench_mp_gap.json.gz to /usr/local/lib/python3.7/dist-packages/matminer/datasets/matbench_mp_gap.json.gz


100%|██████████| 106113/106113 [1:46:35<00:00, 16.59it/s]



matbench_mp_gap parsed!

Fetching matbench_mp_is_metal.json.gz from https://ml.materialsproject.org/projects/matbench_mp_is_metal.json.gz to /usr/local/lib/python3.7/dist-packages/matminer/datasets/matbench_mp_is_metal.json.gz


100%|██████████| 106113/106113 [1:46:35<00:00, 16.59it/s]



matbench_mp_is_metal parsed!

Fetching matbench_mp_e_form.json.gz from https://ml.materialsproject.org/projects/matbench_mp_e_form.json.gz to /usr/local/lib/python3.7/dist-packages/matminer/datasets/matbench_mp_e_form.json.gz


100%|██████████| 132752/132752 [2:08:06<00:00, 17.27it/s]



matbench_mp_e_form parsed!

